In [2]:
from sklearn.cluster import KMeans

In [4]:
import folium
import pandas as pd

In [5]:
df=pd.read_excel('C:/Users/Shiva/Desktop/Canada.xlsx')

In [6]:
df.head()

,Postcode,Latitude,Borough,Neighborhood,Longitude
0,M1B,43.806686,Scarborough,"['Rouge', 'Malvern']",-79.194353
1,M1C,43.784535,Scarborough,"['Highland Creek', 'Rouge Hill', 'Port Union']",-79.160497
2,M1E,43.763573,Scarborough,"['Guildwood', 'Morningside', 'West Hill']",-79.188711
3,M1G,43.770992,Scarborough,['Woburn'],-79.216917
4,M1H,43.773136,Scarborough,['Cedarbrae'],-79.239476


In [21]:
df.at[41,'Borough']='East Toronto'
df.at[42,'Borough']='East Toronto'

In [28]:
dt=df[df['Borough'].str.contains('Toronto')]
dt.head()

,Postcode,Latitude,Borough,Neighborhood,Longitude
37,M4E,43.676357,East Toronto,['The Beaches'],-79.293031
41,M4K,43.679557,East Toronto,"['The Danforth West', 'Riverdale']",-79.352188
42,M4L,43.668999,East Toronto,"['The Beaches West', 'India Bazaar']",-79.315572
43,M4M,43.659526,East Toronto,['Studio District'],-79.340923
44,M4N,43.728020,Central Toronto,['Lawrence Park'],-79.388790


In [26]:
toronto_map=folium.Map(location=[43.676357,-79.293031],zoom_start=10)

In [51]:

folium.features.CircleMarker(
location=[43.676357,-79.293031],
    popup='Toronto',
    fill_color='red',
    radius=10,
).add_to(toronto_map)

In [52]:
toronto_map

In [58]:
for lat,lon,bor,nei in zip(df.Latitude,df.Longitude,df.Borough,df.Neighborhood):
    lab="{},{}".format(nei,bor)
    lab=folium.Popup(lab,parse_html=True)
    folium.features.CircleMarker(
    location=[lat,lon],
        popup=lab,
        fill_color='#FF333F',
        fill_opacity=0.6
    ).add_to(toronto_map)

In [59]:
toronto_map

In [66]:
df1=df.drop('Neighborhood',axis=1)
df1.head()

,Postcode,Latitude,Borough,Longitude
0,M1B,43.806686,Scarborough,-79.194353
1,M1C,43.784535,Scarborough,-79.160497
2,M1E,43.763573,Scarborough,-79.188711
3,M1G,43.770992,Scarborough,-79.216917
4,M1H,43.773136,Scarborough,-79.239476


In [75]:
df2=pd.get_dummies(df1['Borough'])
df2.head()

,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,1,0,0


In [81]:
df2.shape[0]

103

In [83]:
k=df1[['Latitude','Longitude']]
k.shape[0]

103

In [84]:
df3=pd.merge(left=k,right=df2,left_index=True,right_index=True)

In [85]:
df3.head()

,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
0,43.806686,-79.194353,0,0,0,0,0,0,0,0,1,0,0
1,43.784535,-79.160497,0,0,0,0,0,0,0,0,1,0,0
2,43.763573,-79.188711,0,0,0,0,0,0,0,0,1,0,0
3,43.770992,-79.216917,0,0,0,0,0,0,0,0,1,0,0
4,43.773136,-79.239476,0,0,0,0,0,0,0,0,1,0,0


In [87]:
k=KMeans(n_clusters=3,random_state=0).fit(df3)

In [89]:
pred=k.labels_

In [90]:
df3["pred"]=pred

In [94]:
df3.groupby(['pred']).mean()

,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
pred,,,,,,,,,,,,,
0,43.730534,-79.329652,0.25,0.000000,0.000000,0.138889,0.000000,0.000000,0.0,0.000000,0.472222,0.000000,0.138889
1,43.657338,-79.433359,0.00,0.473684,0.131579,0.000000,0.315789,0.026316,0.0,0.026316,0.000000,0.026316,0.000000
2,43.734363,-79.433506,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000


In [95]:
df3.groupby(['pred']).sum()

,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York
pred,,,,,,,,,,,,,
0,1574.299211,-2855.867458,9,0,0,5,0,0,0,0,17,0,5
1,1658.978857,-3018.467625,0,18,5,0,12,1,0,1,0,1,0
2,1268.296529,-2303.571667,0,0,0,0,0,0,29,0,0,0,0


In [103]:
d0=df3[df3["pred"]==0]

In [104]:
d1=df3[df3["pred"]==1]
d2=df3[df3["pred"]==2]

In [105]:
d0.head()

,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Queen's Park,Scarborough,West Toronto,York,pred
0,43.806686,-79.194353,0,0,0,0,0,0,0,0,1,0,0,0
1,43.784535,-79.160497,0,0,0,0,0,0,0,0,1,0,0,0
2,43.763573,-79.188711,0,0,0,0,0,0,0,0,1,0,0,0
3,43.770992,-79.216917,0,0,0,0,0,0,0,0,1,0,0,0
4,43.773136,-79.239476,0,0,0,0,0,0,0,0,1,0,0,0


In [121]:
clus_can=folium.Map(zoom_start=10)

In [122]:
for latitude,longitude in zip(d0.Latitude,d0.Longitude):
    folium.features.CircleMarker(
    location=[latitude,longitude],
        color='#FF333F',
        label="Neighborhood 1",
        fill_opacity=0.4).add_to(clus_can)
for latitude,longitude in zip(d1.Latitude,d1.Longitude):
    folium.features.CircleMarker(
    location=[latitude,longitude],
        color='#6BFF33',
        label="Neighborhood 2",
        fill_opacity=0.4).add_to(clus_can)
for latitude,longitude in zip(d2.Latitude,d2.Longitude):
    folium.features.CircleMarker(
    location=[latitude,longitude],
        color='Yellow',
        label="Neighborhood 3",
        fill_opacity=0.4).add_to(clus_can)

In [123]:
clus_can